<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - Popular nDCG</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender
from lenskit.algorithms.basic import Popular
from lenskit.metrics.predict import rmse
import pandas as pd
import numpy as np

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer():
    all_recs = []
    test_data = []
    pop = Popular(selector=None) # define algorithm

    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('Popular', pop, train, test))

    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)

    rla = topn.RecListAnalysis()
    rla.add_metric(topn.ndcg)
    ndcg = rla.compute(all_recs, test_data)
    return ndcg['ndcg'].mean()

In [3]:
%%time
array = []
for i in range(1,100,1):
    array.append(model_trainer())

Wall time: 1h 9min 8s


In [4]:
metric = pd.DataFrame(array)
metric.columns = ['nDCG']
metric.head(5)

,nDCG
0,0.214048
1,0.216954
2,0.216681
3,0.217680
4,0.218236


In [5]:
metric.sort_values(by=['nDCG'],ascending=False).head(10)

,nDCG
18,0.218436
4,0.218236
64,0.217975
27,0.217954
29,0.217927
72,0.217823
66,0.217794
61,0.217718
54,0.217703
3,0.217680


In [6]:
metric.to_csv('metric_nDCG.csv', index=False)

In [19]:
all_recs.item.unique()

array([288,   1,  56,  98, 117, 222, 313,  69, 173, 748,  50, 258, 181,
       294, 174, 127,   7, 100, 286, 121, 300, 237, 172, 204,  79, 210,
       405, 151,   9, 423, 168, 318,  15, 216,  25, 269, 118, 195, 302,
       257,  22, 234,  89, 328,  96, 276, 176], dtype=int64)

In [21]:
 ratings.groupby("item").count()["rating"].sort_values(ascending=False).head(10)

item
50     583
258    509
100    508
181    507
294    485
286    481
288    478
1      452
300    431
121    429
174    420
127    413
56     394
7      392
98     390
237    384
117    378
172    367
222    365
313    350
204    350
405    344
79     336
210    331
151    326
173    324
69     321
168    316
748    316
269    315
257    303
195    301
423    300
9      299
318    298
276    298
302    297
22     297
96     295
328    295
25     293
15     293
118    293
183    291
216    290
176    284
64     283
Name: rating, dtype: int64